## Chunking

! pip install transformers

In [ ]:
model_name = 'snowflake/snowflake-arctic-embed-m-long'
emb_dimensions = 768

In [ ]:
# Load tokenizer (must match the model)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
filename = "bfp-a3447q.pdf"
input_path = "data/" + filename.split('.')[0]+'.txt'
output_path= input_path.split('.')[0]+'_chunked.txt'
image_path = "data/images"

In [ ]:
import math
def chunk_text_by_lines(text, tokenizer, token_limit):
    """
    Splits text into chunks based on token count, ensuring cuts only happen
    between lines (preserving \n).
    """
    lines = text.splitlines(keepends=True)  # keep '\n' at end of each line
    chunks, current_chunk, current_tokens = [], [], 0
    all_tokens = tokenizer.encode(text, add_special_tokens=False)
    # setting 'soft limit'
    chunk_size = math.ceil(len(all_tokens) / math.ceil(len(all_tokens)/ token_limit))
    for line in lines:
        line_tokens = tokenizer.encode(line, add_special_tokens=False)
        if current_tokens + len(line_tokens) > token_limit:
            # flush current chunk
            chunks.append("".join(current_chunk).strip())
            current_chunk, current_tokens = [], 0
        current_chunk.append(line)
        current_tokens += len(line_tokens)
        if current_tokens > chunk_size:
            # flush current chunk
            chunks.append("".join(current_chunk).strip())
            current_chunk, current_tokens = [], 0
    
    # add last chunk
    if current_chunk:
        chunks.append("".join(current_chunk).strip())
    
    return chunks, chunk_size

In [ ]:
import re
import pathlib, json
json_read = pathlib.Path(input_path).read_text()
data = json.loads(json_read)
chunked_data = []
chunks = []
token_limit = 1000
print(f'Token threshold: {token_limit}')
for index, chunk in enumerate(data):
    text = chunk[-1]
    # Replace <br> with semicolons or newlines to reduce token consumption and clean tables
    text = re.sub(r"<br\s*/?>", "; ", text)
    
    # Strip off excessive whitespace
    text = re.sub(r" +", " ", text)
    text = re.sub(r"\.+", ".", text)
    text = re.sub(r"~", "", text)
        
    chunks, size = chunk_text_by_lines(text, tokenizer, token_limit)
    for text in chunks:
        chunked_data.append(
            [chunk[0],
             chunk[1],
             chunk[2],
             text]
        )
    if len(chunks) > 1:
        print(f"{index}) Text length: {len(text)}\tToken chunk size: {size}\tChunks count: {len(chunks)}\t for {chunk[1]}")


In [ ]:
import json
json_file=json.dumps(chunked_data, indent=2)
# Exporting data to output file for storage
with open(output_path, mode='w+') as f_out:
    f_out.write(json_file)

In [ ]:
data_folder = "data/"
filename = data_folder + "bfp-a3447q.pdf"
content_path= filename.split('.')[0]+'_chunked.txt'
context_path= filename.split('.')[0]+'_context.txt'
image_path = "data/images"

In [ ]:
import pathlib, json
json_read = pathlib.Path(context_path).read_text()
data_context = json.loads(json_read)
data_context = {int(k): v for k, v in data_context.items()}

In [ ]:
max_len = 0
for k,v in data_context.items():
    all_tokens = tokenizer.encode(v, add_special_tokens=False)
    if len(all_tokens) > 600:
        if len(all_tokens) > max_len:
            max_len = len(all_tokens)
        print(f'For chapter no: {k}, token count is {len(all_tokens)}')

In [ ]:
text = """
The document provides technical specifications for different robot models (RH-6CH6020-S11/S15, RH-6CH7020-S11/S15, RH-6CH6020-S23/S24, RH-6CH7020-S23/S24). These specifications include allowable inertia, J3(Z) axis pressing force, maximum eccentricity, mass, tool wiring , tool pneumatic pipes, supply pressure, protection, ambient temperature, painting color, and machine cable length.\nNotes clarify: Note 1 covers max speed with MvTune2 and no auto compensation. Note 3 explains cycle time under MvTune2, affected by accuracy needs and carrying mass per model. Note 5 defines J3(Z) axis pressing force and overload risks. Note 8 explains setting environmental temperature with parameter OLTMX for control and protection.\n\nSection 2.1.2 addresses counter-force on installation surface, listing falls moment (ML), torsion moment (MT), horizontal force (FH), and vertical force (FV) for RH-3CH-Sxx, RH-6CH6020-Sxx, and RH-6CH7020-Sxx.\n\nSection 2.2 covers mass capacity and tooling: eccentric loads impose restrictions; tooling must follow allowable inertia values (2.1.1). Cautions include alarms (vibration, overload, overcurrent) caused by speed or posture even within limits; proper parameter settings (mass, centroid) are crucial to prevent gear/belt wear. Mass capacity and allowable inertia are dynamic limits, not guarantees of accuracy. Long or low-rigid tooling may reduce accuracy or cause vibration. Allowable offset (Z direction) from shaft to center of gravity is 100 mm. Load deviation allowance is 150 mm, reduced to 10 mm in large inertia mode. Figures 2-1 to 2-3 show center of gravity for loads.\n\nSection 2.2.3 explains mass capacity, speed, and acceleration/deceleration: robot optimizes automatically based on load data (mass and size). Manual reduction of speed/accel may be required if vibration or overheating occurs. Wrong load settings shorten lifespan. For high accuracy, correct load input and reduced acceleration/deceleration are recommended. Parameters: "HNDDAT*" (hand), "WRKDAT*" (work), set with "LoadSet" command (default LoadSet 0.0). Factory defaults for hand mass, size, and center of gravity are given in a table.
"""
all_tokens = tokenizer.encode(text, add_special_tokens=False)

In [ ]:
len(all_tokens)

In [ ]:
text

In [ ]:
data_context[34] = text

In [ ]:
import json
json_file=json.dumps(data_context, indent=2)
# Exporting data to output file for storage
with open(context_path, mode='w+') as f_out:
    f_out.write(json_file)